In [5]:
!pip install hmmlearn==0.3.0
!pip install numpy==1.24.3
!pip install scikit-learn===1.2.1
!pip install konlpy===0.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 72.1 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.33.0 requires scikit-learn>=1.2.2, but you have scikit-learn 1.2.1 which is incompatible.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.1 which is incompatible.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.2.1 which is incompatible.


In [7]:
import random
from hmmlearn import hmm
import numpy as np
from sklearn.preprocessing import LabelEncoder
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer

# 한글 형태소 분석기 초기화
okt = Okt()

# 치킨 메뉴 리스트
chicken_menus = ["BBQ", "페리카나", "BHC", "교촌"]

# 확장된 데이터셋 준비: 사용자 입력 문장과 대응하는 상태(의도)
inputs = [
    "안녕", "안녕하세요", "하이", "반가워요", "좋은 하루에요",
    "치킨 추천해줘", "뭐 먹을까?", "오늘 뭐 먹지?", "치킨 추천", "배고파 치킨 먹고 싶어",
    "종료", "그만", "끝내자", "종료하고 싶어", "대화 끝",
    "다시 추천해줘", "또 추천해줘", "더 추천해줘", "다른 치킨 추천해줘", "색다른 메뉴 추천",
    "치킨 종류 알려줘", "어떤 치킨 있어?", "메뉴 보여줘", "선택할 수 있는 치킨?", "추천 가능한 치킨 종류는?",
    "BBQ 먹고 싶어", "BBQ 어때?", "BBQ 맛있어?", "BBQ 치킨", "BBQ 치킨 추천",
    "매운 치킨 추천", "매운 맛으로 추천해줘", "매운 치킨 뭐 있어?", "얼얼한 치킨 추천", "매운 음식 좋아해",
    "가장 인기 있는 치킨은?", "제일 인기 있는 치킨은?", "인기 메뉴 알려줘", "사람들이 많이 먹는 치킨은?", "추천 많이 받는 치킨?",
    "교촌 어때?", "교촌 추천", "교촌이 맛있어?", "교촌 치킨", "교촌 치킨 추천",
    "페리카나가 좋아", "페리카나 추천해줘", "페리카나 치킨", "페리카나 메뉴는?", "페리카나 어때?",
    "BHC 추천해줘", "BHC 치킨 어때?", "BHC 메뉴", "BHC 추천", "BHC가 좋아",
    "양념 치킨 먹고 싶어", "양념 치킨 추천", "달콤한 치킨 추천", "양념이 맛있는 치킨?", "양념으로 추천해줘",
    "후라이드 어때?", "후라이드 추천", "후라이드 맛있어?", "기본 치킨 추천", "오리지널 치킨 추천",
    "치킨 주문", "치킨 주문할게", "주문 도와줘", "치킨 배달", "치킨 시켜줘"
]
states = [
    "GREETING", "GREETING", "GREETING", "GREETING", "GREETING",
    "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION",
    "EXIT", "EXIT", "EXIT", "EXIT", "EXIT",
    "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION",
    "INQUIRY", "INQUIRY", "INQUIRY", "INQUIRY", "INQUIRY",
    "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION",
    "SPICY", "SPICY", "SPICY", "SPICY", "SPICY",
    "POPULAR", "POPULAR", "POPULAR", "POPULAR", "POPULAR",
    "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION",
    "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION",
    "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION",
    "SPICY", "SPICY", "SPICY", "SPICY", "SPICY",
    "INQUIRY", "INQUIRY", "INQUIRY", "INQUIRY", "INQUIRY",
    "ORDER", "ORDER", "ORDER", "ORDER", "ORDER"
]

# 한글 전처리 및 토큰화 함수
def preprocess_korean_input(text):
    tokens = okt.morphs(text)
    return ' '.join(tokens)

# 텍스트 전처리를 위한 CountVectorizer 사용 (한글 토큰화 반영)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform([preprocess_korean_input(text) for text in inputs])

# 라벨 인코더로 텍스트 데이터를 숫자로 변환
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(states)

# HMM 모델 초기화 및 훈련 부분 수정
unique_states = list(set(states))
n_states = len(unique_states)
n_features = X.shape[1]

# 상태 매핑 생성
state_to_idx = {state: idx for idx, state in enumerate(unique_states)}
state_sequences = np.array([state_to_idx[state] for state in states])

# 데이터 준비
X_int = X.toarray().astype(int)
n_trials = 1

# 각 상태별 단어 출현 빈도 계산 (라플라스 스무딩 적용)
emissionprob = np.ones((n_states, n_features))  # 1로 초기화 (스무딩)
state_counts = np.ones(n_states)  # 1로 초기화 (스무딩)

# 각 상태별로 단어 출현 빈도 누적
for i in range(len(state_sequences)):
    state = state_sequences[i]
    state_counts[state] += 1
    emissionprob[state] += X_int[i]

# 각 상태별로 정규화
for i in range(n_states):
    emissionprob[i] = emissionprob[i] / emissionprob[i].sum()

# 전이 확률에도 스무딩 적용
transmat = np.ones((n_states, n_states))  # 1로 초기화
for i in range(len(state_sequences)-1):
    curr_state = state_sequences[i]
    next_state = state_sequences[i+1]
    transmat[curr_state][next_state] += 1

# 전이 확률 정규화
for i in range(n_states):
    transmat[i] = transmat[i] / transmat[i].sum()

# 시작 확률에도 스무딩 적용
startprob = np.ones(n_states)
for state in state_sequences:
    startprob[state] += 1
startprob = startprob / startprob.sum()

# MultinomialHMM 모델 초기화
model = hmm.MultinomialHMM(
    n_components=n_states,
    n_iter=1,
    init_params="",
)
model.n_trials = n_trials
model.startprob_ = startprob
model.transmat_ = transmat
model.emissionprob_ = emissionprob

def predict_state(user_input):
    """사용자 입력을 받아 상태를 예측합니다."""
    try:
        # 입력 텍스트 전처리
        user_input_processed = preprocess_korean_input(user_input)

        # 벡터화
        input_vector = vectorizer.transform([user_input_processed]).toarray().astype(int)

        # 입력 정규화
        input_normalized = np.zeros_like(input_vector)
        if input_vector.sum() > 0:
            max_idx = np.argmax(input_vector[0])
            input_normalized[0, max_idx] = n_trials
        else:
            input_normalized[0, 0] = n_trials

        # 상태 예측
        logprob, state_sequence = model.decode(input_normalized)
        predicted_state = unique_states[state_sequence[0]]

        # 디버깅을 위한 자세한 정보 출력
        # print(f"\n디버그 - 입력: {user_input}")
        # print(f"전처리된 텍스트: {user_input_processed}")
        # print(f"입력 벡터: {input_vector}")
        # print(f"정규화된 입력: {input_normalized}")
        # print(f"예측된 상태: {predicted_state}")

        # 각 상태의 확률 출력
        # state_probs = model.predict_proba(input_normalized)[0]
        # for state, prob in zip(unique_states, state_probs):
        #     print(f"{state} 상태의 확률: {prob:.4f}")

        return predicted_state
    except Exception as e:
        print(f"예측 중 오류 발생: {str(e)}")
        return "UNKNOWN"

def choice_chicken():
    print("안녕하세요! 저는 치킨 메뉴를 추천해주는 챗봇입니다.")
    name = input("우선, 당신의 이름을 알려주세요: ")
    print(f"반갑습니다, {name}님! 저는 다양한 치킨 메뉴를 추천해드릴 수 있어요.")

    while True:
        user_input = input("\n어떤 치킨을 추천해드릴까요? (종료하려면 '종료'라고 입력): ")

        if user_input.lower() == "종료":
            print("대화를 종료합니다. 맛있게 드세요~")
            break

        # 사용자 입력에 따라 상태 예측
        predicted_state = predict_state(user_input)
        print(f" 입력: {user_input}, 예측된 상태: {predicted_state}")  # 디버깅용 출력

        # 예측된 상태에 따라 응답 생성
        if predicted_state == "GREETING":
            print("안녕하세요! 무엇을 도와드릴까요?")
        elif predicted_state == "RECOMMENDATION":
            recommended_menu = random.choice(chicken_menus)
            print(f"{recommended_menu} 치킨을 추천드려요! 맛있게 드세요, {name}님!")
        elif predicted_state == "SPICY":
            print(f"매운 치킨을 원하신다면 {random.choice(['BHC', '교촌'])}의 매운 치킨을 추천드려요!")
        elif predicted_state == "POPULAR":
            print("가장 인기 있는 치킨은 BBQ와 교촌입니다. 어떤 걸 드실래요?")
        elif predicted_state == "INQUIRY":
            print(f"치킨 종류는 {', '.join(chicken_menus)}가 있습니다. 무엇을 드시고 싶으신가요?")
        elif predicted_state == "ORDER":
            print("주문을 도와드리겠습니다! 어떤 치킨을 원하시나요?")
        else:
            print("죄송해요, 잘 이해하지 못했어요. 다시 말씀해주시겠어요?")

# Choice_Chicken 챗봇 시작
choice_chicken()

https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


안녕하세요! 저는 치킨 메뉴를 추천해주는 챗봇입니다.
우선, 당신의 이름을 알려주세요: 감성
반갑습니다, 감성님! 저는 다양한 치킨 메뉴를 추천해드릴 수 있어요.

어떤 치킨을 추천해드릴까요? (종료하려면 '종료'라고 입력): 매운 치킨
 입력: 매운 치킨, 예측된 상태: SPICY
매운 치킨을 원하신다면 BHC의 매운 치킨을 추천드려요!

어떤 치킨을 추천해드릴까요? (종료하려면 '종료'라고 입력): 인기있는 치킨
 입력: 인기있는 치킨, 예측된 상태: POPULAR
가장 인기 있는 치킨은 BBQ와 교촌입니다. 어떤 걸 드실래요?


KeyboardInterrupt: Interrupted by user